In [1]:
import importlib.util
import matplotlib.dates as md
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

if importlib.util.find_spec('seaborn') is not None:
    import seaborn as sns
    sns.set()

## Load data

##### Travel time

In [3]:
# - Read in travel time data
df = pd.read_csv('/opt/dssg-hot/data/travel_times/concatenated_405_travel_times_5am_to_8pm_5min.csv')

# - Replace -86400 w/ nan
df.replace(-86400, np.nan, inplace=True)

# - Set 'Time' column to index
df.set_index('Time',inplace=True)

# - Compute GP minus HOT travel time differences and add diff as new columns
colsnow = df[df.columns[::2]].columns
dfgp = df[df.columns[::2]]
dfhov = df[df.columns[1::2]]
dfhov.columns = colsnow
dfgmh = dfgp-dfhov # GP minus HOV = gmh
dfgmh.columns = dfgmh.columns.str.replace('GP','GMH')
df = pd.concat([df, dfgmh], axis=1)

# - Unset 'Time' column to index
df.reset_index(inplace=True)

# - Examine data
df.head(2)

                     GP_12_12  HOV_12_12  GP_11_11  HOV_11_11  GP_11_12  \
Time                                                                      
2018-01-01 05:00:00      27.0       27.0      34.2       34.2      61.2   
2018-01-01 05:05:00      27.0       27.0      34.2       34.2      61.2   

                     HOV_11_12  GP_14_14  HOV_14_14  GP_14_15  HOV_14_15  ...  \
Time                                                                      ...   
2018-01-01 05:00:00       61.2      76.8       76.8     136.2      136.2  ...   
2018-01-01 05:05:00       61.2      76.8       76.8     136.2      136.2  ...   

                     GP_16_23  HOV_16_23  GP_23_23  HOV_23_23  GP_21_23  \
Time                                                                      
2018-01-01 05:00:00     503.4      503.4     102.6      102.6     132.0   
2018-01-01 05:05:00     503.4      503.4     102.6      102.6     132.0   

                     HOV_21_23  GP_20_23  HOV_20_23  GP_19_23  HOV_19_23 

## Add and groupby useful computed time columns

In [4]:
df['new_dates'] = pd.to_datetime(df['Time'])
df['weekday_num'] = df['new_dates'].apply(lambda t: t.weekday())
df['month_num'] = df['new_dates'].apply(lambda t: t.month)
df['hour_num'] = df['new_dates'].apply(lambda t: t.hour)
df['time_of_day'] = df['new_dates'].apply(lambda t: t.time())
# NOTE: mon = 0, fri = 4; jan = 1, dec = 12

In [4]:
weekday_num_grouped = df.groupby('weekday_num')
weekday_num_stds = weekday_num_grouped.aggregate(np.std)
weekday_num_meds = weekday_num_grouped.aggregate(np.median)
weekdayxtimeofday_grouped = df.groupby(['weekday_num','time_of_day'])
weekdayxtimeofday_stds = weekdayxtimeofday_grouped.aggregate(np.std)
weekdayxtimeofday_meds = weekdayxtimeofday_grouped.aggregate(np.median)

## Define fxns to plot travel times in diff ways

#### Plot travel times and travel time savings over the day by month

In [5]:
def plot_tt_over_day_by_month(tt_or_tts, entry_plaza, exit_plaza):
    entry_plaza_str = str(entry_plaza).zfill(2)
    exit_plaza_str = str(exit_plaza).zfill(2)
    entry_exit_plaza_str = entry_plaza_str + '_' + exit_plaza_str
    
    fig, axs = plt.subplots(4, 3, figsize=(16,12), sharex=True, sharey=True)

    #ymin = np.nanmin(df[['GP_' + entry_exit_plaza_str, 'HOV_' + entry_exit_plaza_str]])/60
    #ymax = np.nanmax(df[['GP_' + entry_exit_plaza_str, 'HOV_' + entry_exit_plaza_str]])/60

    #xminorlocator = md.MinuteLocator(byminute=[0,15,30,45], interval = 1)
    #xmajorlocator = md.HourLocator() # every hour

    #bigax = fig.add_subplot(111)
    #bigax.set_xlabel('Time of day (hour)')
    #bigax.set_ylabel('Travel time (minutes)')
    #fig.suptitle('Each month ' + entry_plaza_str + '_' + exit_plaza_str + ' travel times')

    isp_x = 0; isp_y = -1;
    for month in range(1,13):
        if (month!=5) and (month!=9):
            isp_y = isp_y + 1;
        elif month==5:
            isp_y = 0;
            isp_x = 1;
        elif month==9:
            isp_y = 0;
            isp_x = 2;

        if tt_or_tts=='tt':
            #axs[isp_y][isp_x].set_ylim([ymin,ymax])
            axs[isp_y][isp_x].plot(
                df[df['month_num']==month]['time_of_day'],
                df[df['month_num']==month]['GP_' + entry_exit_plaza_str]/60,
                'C1-')
            axs[isp_y][isp_x].plot(
                df[df['month_num']==month]['time_of_day'],
                df[df['month_num']==month]['HOV_' + entry_exit_plaza_str]/60,
                'C2-')
        elif tt_or_tts=='tts':
            axs[isp_y][isp_x].plot(
                df[df['month_num']==month]['time_of_day'],
                df[df['month_num']==month]['GMH_' + entry_exit_plaza_str]/60)
        
        axs[isp_y][isp_x].set_title('Month number: %i' %month)
        #axs[isp_y][isp_x].xaxis.set_major_locator(xmajorlocator)
        #axs[isp_y][isp_x].xaxis.set_minor_locator(xminorlocator)
        axs[isp_y][isp_x].yaxis.grid(True)
        axs[isp_y][isp_x].xaxis.grid(True)
        axs[isp_y][isp_x].tick_params(axis='x', labelrotation=90)

#### Plot travel times and travel time savings over the day by day of the week

In [6]:
def plot_tt_over_day_by_day_of_week(tt_or_tts, entry_plaza, exit_plaza):
    entry_plaza_str = str(entry_plaza).zfill(2)
    exit_plaza_str = str(exit_plaza).zfill(2)
    entry_exit_plaza_str = entry_plaza_str + '_' + exit_plaza_str
    
    fig, axs = plt.subplots(2, 3, figsize=(16,6), sharex=True, sharey=True)
    isp_x = 0; isp_y = -1;
    for day in range(0,6): # plot friday twice
        if (day!=2) and (day!=4):
            isp_y = isp_y + 1;
        elif day==2:
            isp_y = 0;
            isp_x = 1;
        elif day==4:
            isp_y = 0;
            isp_x = 2;

        if day==5: # plot friday twice
            day = 4;

        if tt_or_tts=='tt':
            axs[isp_y][isp_x].plot(
                df[df['weekday_num']==day]['time_of_day'],
                df[df['weekday_num']==day]['GP_' + entry_exit_plaza_str]/60,
                'C1-')
            axs[isp_y][isp_x].plot(
                df[df['weekday_num']==day]['time_of_day'],
                df[df['weekday_num']==day]['HOV_' + entry_exit_plaza_str]/60,
                'C2-')
        elif tt_or_tts=='tts':
            axs[isp_y][isp_x].plot(
                df[df['weekday_num']==day]['time_of_day'],
                df[df['weekday_num']==day]['GMH_' + entry_exit_plaza_str]/60)
        
        axs[isp_y][isp_x].set_title('Week day number: %i' %day)
        axs[isp_y][isp_x].yaxis.grid(True)
        axs[isp_y][isp_x].xaxis.grid(True)
        axs[isp_y][isp_x].tick_params(axis='x', labelrotation=90)

#### Plot travel times and travel time savings over the day by day of the week w/ medians and stdevs

In [7]:
def plot_tt_over_day_by_day_of_week_medstd(tt_or_tts, entry_plaza, exit_plaza):
    entry_plaza_str = str(entry_plaza).zfill(2)
    exit_plaza_str = str(exit_plaza).zfill(2)
    entry_exit_plaza_str = entry_plaza_str + '_' + exit_plaza_str

    fig, axs = plt.subplots(2, 3, figsize=(16,6), sharex=True, sharey=True)
    isp_x = 0; isp_y = -1;
    for day in range(0,6): # plot friday twice
        if (day!=2) and (day!=4):
            isp_y = isp_y + 1;
        elif day==2:
            isp_y = 0;
            isp_x = 1;
        elif day==4:
            isp_y = 0;
            isp_x = 2;

        if day==5: # plot friday twice
            day = 4;

        if tt_or_tts=='tt':
            axs[isp_y][isp_x].plot(
                (weekdayxtimeofday_meds['GP_' + entry_exit_plaza_str][day]+weekdayxtimeofday_stds['GP_' + entry_exit_plaza_str][day])/60,'C1--')
            axs[isp_y][isp_x].plot(
                weekdayxtimeofday_meds['GP_' + entry_exit_plaza_str][day]/60,'C1-')
            axs[isp_y][isp_x].plot(
                (weekdayxtimeofday_meds['GP_' + entry_exit_plaza_str][day]-weekdayxtimeofday_stds['GP_' + entry_exit_plaza_str][day])/60,'C1--')
            axs[isp_y][isp_x].plot(
                (weekdayxtimeofday_meds['HOV_' + entry_exit_plaza_str][day]+weekdayxtimeofday_stds['HOV_' + entry_exit_plaza_str][day])/60,'C2--')
            axs[isp_y][isp_x].plot(
                weekdayxtimeofday_meds['HOV_' + entry_exit_plaza_str][day]/60,'C2-')
            axs[isp_y][isp_x].plot(
                (weekdayxtimeofday_meds['HOV_' + entry_exit_plaza_str][day]-weekdayxtimeofday_stds['HOV_' + entry_exit_plaza_str][day])/60,'C2--')
        elif tt_or_tts=='tts':
            axs[isp_y][isp_x].plot(
                (weekdayxtimeofday_meds['GMH_' + entry_exit_plaza_str][day]+weekdayxtimeofday_stds['GMH_' + entry_exit_plaza_str][day])/60,'k--')
            axs[isp_y][isp_x].plot(
                weekdayxtimeofday_meds['GMH_' + entry_exit_plaza_str][day]/60,'k-')
            axs[isp_y][isp_x].plot(
                (weekdayxtimeofday_meds['GMH_' + entry_exit_plaza_str][day]-weekdayxtimeofday_stds['GMH_' + entry_exit_plaza_str][day])/60,'k--')            
            
        axs[isp_y][isp_x].set_title('Week day number: %i' %day)
        axs[isp_y][isp_x].tick_params(axis='x', labelrotation=90)